In [78]:
import numpy as np

# Function: Creates the match matrix for use to calculate the score of each match
# Inputs:
#   num: number of frames to consider
# Returns: 
#   match: match matrix
def make_match_matrix(num):
    match = np.zeros((num, num))
    for i in range(0,num):
        for j in range(0,num):
            if i == j:
                match[i,j] = 1
            else:
                match[i,j] = 1 - abs((i - j))/float(10)
    return match

# Function: Create the matrix M, Ix, and Iy
# Inputs:
#   A: sequence A 
#   B: sequence B
#   gap_penalty: array containing the gap penalty for A and B
#   mrow: letters in alphabet for sequence A
#   mcol: letters in alphabet for sequence B
#   match: match matrix
#   M: Alignment matrix M (also contains the pointers)
#   Ix: Alignment matrix Ix (also contains the pointers)
#   Iy: Alignment matrix Iy (also contains the pointers)
#   x: length of A + 1
#   y: length of B + 1
#   align_type: boolean; global == True; local == False
# Returns: 
#   matrix: contains M, Ix, and Iy. M = [M, Ix, and Iy].
#           each matrix is two layered to also store the pointers
def make_M_Ix_Iy(A, B, gap_penalty, mrow, mcol, match, x, y, align_type):
    
    matrix = []
    for i in range(3): #to store matrix M, Ix, and Iy
        matrix.append(np.zeros((2, x, y)))

    for i in range(1,x):
        for j in range(1, y):
            match_i = A[i-1]
            match_j = B[j-1]
            score = round(match[match_i, match_j],2)
            fill_M(matrix[0], matrix[1], matrix[2], i, j, score, align_type)
            fill_Ix(matrix[0], matrix[1], i, j, gap_penalty[2], gap_penalty[3], align_type) #dy and ey
            fill_Iy(matrix[0], matrix[2], i, j, gap_penalty[0], gap_penalty[1], align_type) #dx and ey
    return matrix
    
# Function: Fills in matrix M
# Inputs:
#   M: Alignment matrix M (also contains the pointers)
#   Ix: Alignment matrix Ix (also contains the pointers)
#   Iy: Alignment matrix Iy (also contains the pointers)
#   i: specifies current row being examined
#   j: specifies current column being examined
#   align_type: boolean; global == True; local == False
# Returns: 
#   nothing
def fill_M(M, Ix, Iy, i, j, score, align_type):
    a = round(M[0][i - 1][j - 1] + score, 3) #score for M
    b = round(Ix[0][i - 1][j - 1] + score, 3) #score for Ix
    c = round(Iy[0][i - 1][j - 1] + score, 3) #score for Iy
    
    values = np.array([a, b, c])
    M[0][i][j] = np.amax(values)

    if (M[0][i][j] < 0) and (align_type == 0):
        M[0][i][j] = 0
    if a >= b and a >= c: #M is the best
        M[1][i][j] = 1
    if b >= a and b >= c: #Ix is the best
        M[1][i][j] = M[1][i][j]*10 + 2
    if c >= a and c >= b: #Iy is the best
        M[1][i][j] = M[1][i][j]*10 + 3

# Function: Fills in matrix Ix
# Inputs:
#   M: Alignment matrix M (also contains the pointers)
#   Ix: Alignment matrix Ix (also contains the pointers)
#   i: specifies current row being examined
#   j: specifies current column being examined
#   dy: opening gap penalty for B
#   ey: extending gap penalty for B
#   align_type: boolean; global == True; local == False
# Returns: 
#   nothing
def fill_Ix(M, Ix, i, j, dy, ey, align_type):
    a = round(M[0][i - 1][j] - dy, 3)
    b = round(Ix[0][i - 1][j] - ey, 3)
    
    values = np.array([a, b])
    Ix[0][i][j] = np.amax(values)
    
    if (Ix[0][i][j] < 0) and (align_type == 0):
        Ix[0][i][j] = 0
    if a >= b:
        Ix[1][i][j] = 1
    if b >= a:
        Ix[1][i][j] = Ix[1][i][j]*10 + 2

# Function: Fills in matrix Iy
# Inputs:
#   M: Alignment matrix M (also contains the pointers)
#   Iy: Alignment matrix Ix (also contains the pointers)
#   i: specifies current row being examined
#   j: specifies current column being examined
#   dx: opening gap penalty for A
#   ex: extending gap penalty for A
#   align_type: boolean; global == True; local == False
# Returns: 
#   nothing
def fill_Iy(M, Iy, i, j, dx, ex, align_type):
    a = round(M[0][i][j - 1] - dx, 3)
    b = round(Iy[0][i][j - 1] - ex, 3)

    values = np.array([a, b])
    Iy[0][i][j] = np.amax(values)

    if (Iy[0][i][j] < 0) and (align_type == 0):
        Iy[0][i][j] = 0
    if a >= b:
        Iy[1][i][j] = 1
    if b >= a:
        Iy[1][i][j] = Iy[1][i][j]*10 + 3

# Function: Finds the starting point for sequence alignment and conducts it
# Inputs:
#   A: sequence A 
#   B: sequence B
#   matrix: contains alignment matrix M, Ix, and Iy
#   x: length of A + 1
#   y: length of B + 1
#   align_type: boolean; global == True; local == False
# Returns: 
#   result: contains all the alignment results and alignment score
def get_sequence(A, B, matrix, x, y, align_type):
    M = matrix[0]
    if align_type:
        end = np.concatenate([M[0][x,:y], np.flipud(M[0][:,y])])
        seed = np.argwhere(end == np.amax(end)).flatten().tolist()
    else:
        end = M[0][:][:]
        seed = np.argwhere(end == np.amax(end)).flatten().tolist()
    
    result = []
    result.append(np.amax(end))

    if align_type:
        for i in seed:
            row = x*(i <= x) + (i > x)*(y - (i - x))
            col = i*(i <= x) + (i > x)*y
            traceback(0, matrix, row, col, '', '', A, B, result, align_type)
    else:
        for i in range(0,len(seed)/2):
            row = seed[2*i]
            col = seed[2*i + 1]
            traceback(0, matrix, row, col, '', '', A, B, result, align_type)
    return result

# Function: Sequence traceback
# Inputs:
#   num: indicates which matrix the traceback is currently on
#       0 == M, 1 == Ix, 2 == Iy
#   matrix: contains alignment matrix M, Ix, and Iy
#   i: current row 
#   j: current col
#   a: sequence A alignment build up
#   b: sequence B alignment build up
#   A: sequence A 
#   B: sequence B
#   align_type: boolean; global == True; local == False
# Returns: 
#   result: contains all the alignment results and alignment score
def traceback(num, matrix, i, j, a, b, A, B, result, align_type):
    if align_type:
        base_case = (i == 0 or  j == 0)
    else:
        base_case = (matrix[0][0][i][j] == 0)
        
    if align_type:
        if base_case: #base case
            if num == 0:
                result.append(a)
                result.append(b)
            return
    else:
        if base_case and num == 0: #base case
            result.append(a)
            result.append(b)
            return

    pointer = str(int(matrix[num][1][i][j]))
    if num == 0: #matrix M
        a = str(A[i - 1]) + ' ' + a
        b = str(B[j - 1]) + ' ' + b
        case_1 = (i == 1 or j == 1)
        i -= 1
        j -= 1
    elif num == 1: #matrix Ix
        a = str(A[i - 1]) + ' ' + a
        b = '_' + ' ' + b
        case_1 = (i == 1)
        i -= 1
    elif num == 2: #matrix Iy
        a = '_' + ' ' + a
        b = str(B[j - 1]) + ' ' + b
        case_1 = (j == 1)
        j -= 1

    case_2 = False
    for point in pointer:
        if not align_type:
            case_2 = (int(point) == 1 and matrix[0][0][i][j] == 0)

        traceback(int(point) - 1, matrix, i, j, a, b, A, B, result, align_type)
        if case_1 or case_2:
            break

# Function: Output alignment results to file
# Inputs:
#   filename: the file to be created to store the results
#   results: contains all the alignment results and alignment score
# Returns: 
#   nothing
def output_results(filename, result):
    f = open(filename, 'w')
    print round(result[0],1)
    result = result[1:]

    for i in range(0, len(result)/2):
        print '\n'
        print result[2*i]
        print result[2*i + 1]
    f.close()

# Function: Run sequence alignment
# First it gathers the data from the file and then runs sequence alignment
# Inputs:
#   filename: name for the file that contains information for sequencing
#   output_filename: prints all the results into a file of the same name
# Returns: 
#   nothing
def seq_align(seq_A, seq_B, align_type, open_gap_penalty, extend_gap_penality):
    #gather data from file
    align_type = align_type == 0 #global vs local
    gap_penalty = [open_gap_penalty, extend_gap_penality, open_gap_penalty, extend_gap_penality]
    mrow = np.amax(np.hstack([seq_A ,seq_B])) + 1
    mcol = mrow

    #generate match matrix
    match = make_match_matrix(mrow)
    
    #calculate the M, Ix, and Iy matrix
    x = len(seq_A) + 1
    y = len(seq_B) + 1
    matrix = make_M_Ix_Iy(seq_A, seq_B, gap_penalty, mrow, mcol, match, x, y, align_type)
    
    #do global or local alignment
    result = get_sequence(seq_A, seq_B, matrix, x - 1, y - 1, align_type)
    output_results(output_filename, result)


In [83]:
seq_A = [0,1,2,3,4,5,6,7,8,9,10]
seq_B = [1,3,5,8,9,10,11,12,13,14,10,10,10,4,5,6,7,8,9,10]

align_type = 1 #local alignment == 1; global alignment == 0
open_gap_penalty = 0
extend_gap_penalty = 0.1
seq_align(seq_A, seq_B, align_type, open_gap_penalty, extend_gap_penalty)

9.1


0 1 2 _ 3 _ _ _ _ _ _ _ _ 4 5 6 7 8 9 10 
1 3 5 8 9 10 11 12 13 14 10 10 10 4 5 6 7 8 9 10 


0 1 2 3 _ _ _ _ _ _ _ _ _ 4 5 6 7 8 9 10 
1 3 5 8 9 10 11 12 13 14 10 10 10 4 5 6 7 8 9 10 


In [28]:
A = 'FA'
i = 2
mrow = 'ABCDEF'
mrow.index(A[0])

5